## Local Testing

In [ ]:
import requests
import json
import httpx

In [ ]:
response = requests.post(
    "http://localhost:8000/chat",
    json={"message": "Show me the tables available in ENAHO database"}
)
data = response.json()
print(f"Response: {data['response']}")
print(f"Thread ID: {data['thread_id']}")

In [10]:
response = requests.post(
    "http://localhost:8000/chat",
    json={
        "message": "What columns are in the first table?",
        "thread_id": data['thread_id']
    }
)

In [ ]:
data = response.json()
print(f"Response: {data['response']}")
print(f"Thread ID: {data['thread_id']}")

In [ ]:
url = "http://localhost:8000/chat/stream"
payload = {
    "message": "De acuerdo a la GEIH, ¿qué porcentaje de hogares pagan por alquiler de su vivienda? Desagregado por departamento",
    "thread_id": "user-session-790"
}

async with httpx.AsyncClient(timeout=None) as client:
    async with client.stream("POST", url, json=payload) as response:       
        full_text = ""
        async for line in response.aiter_lines():
            if line.startswith("data: "):
                content = line[len("data: "):].strip()
                content = eval(content)["content"]
                full_text += content
                print(content, end="", flush=True)
        
        print("\n\nFull response:", full_text)

## Cloud Run Testing

In [ ]:
BASE_URL = "https://esma-agent-514700908055.us-east1.run.app"

In [ ]:
# Test basic connectivity
print("Testing root endpoint...")
response = requests.get(f"{BASE_URL}/")
print(json.dumps(response.json(), indent=2))

In [ ]:
# Test a simple query
print("\n\nTesting chat endpoint...")
response = requests.post(
    f"{BASE_URL}/chat",
    json={"message": "List the tables available in ENAHO"}
)
result = response.json()
print(f"Thread ID: {result.get('thread_id')}")
print(f"Response preview: {result.get('response', 'No response')[:500]}...")